In [1]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)


In [2]:
import pandas as pd

data = pd.read_csv('../../src/data/Hyperliquid Purty Palooza Winner List.csv', usecols=['Wallet', 'HARMONIX']).iloc[1:]
print(data.head())

                                       Wallet HARMONIX
1  0x287cDA72b62189a9A2b83a7A369fdFAe8101BED7     0.12
2  0xBC05da14287317FE12B1a2b5a0E1d756Ff1801Aa     0.23
3                                         NaN        0


In [3]:
import eth_account
from eth_account.signers.local import LocalAccount
from core.config import settings
from hyperliquid.exchange import Exchange
from hyperliquid.info import Info


def setup(base_url=None, skip_ws=False):
    account: LocalAccount = eth_account.Account.from_key(settings.HYPER_LIQUID_API_KEY)
    address = settings.HYPER_LIQUID_ADDRESS
    if address == "":
        address = account.address
    print("Running with account address:", address)
    if address != account.address:
        print("Running with agent address:", account.address)
    info = Info(base_url, skip_ws)
    user_state = info.user_state(address)
    spot_user_state = info.spot_user_state(address)
    margin_summary = user_state["marginSummary"]
    if float(margin_summary["accountValue"]) == 0 and len(spot_user_state["balances"]) == 0:
        print("Not running the example because the provided account has no equity.")
        url = info.base_url.split(".", 1)[1]
        error_string = f"No accountValue:\nIf you think this is a mistake, make sure that {address} has a balance on {url}.\nIf address shown is your API wallet address, update the config to specify the address of your account, not the address of the API wallet."
        raise Exception(error_string)
    exchange = Exchange(account, base_url, account_address=address)
    return address, info, exchange

In [9]:
from hyperliquid.utils import constants
address, info, exchange = setup(constants.TESTNET_API_URL, skip_ws=True)
if exchange.account_address != exchange.wallet.address:
    raise Exception("Agents do not have permission to perform internal transfers")

for index, row in data.iterrows():
    #check if row['HARMONIX'] is empty or not, check if it is NaN check if it is None
    if pd.isnull(row['Wallet']) or row['Wallet'] == "None" or row['Wallet'] == "": 
        continue
    transfer_result = exchange.spot_transfer(
        row['HARMONIX'], row['Wallet'], "HYPE:0x7317beb7cceed72ef0b346074cc8e7ab"
    )
    print(transfer_result)


Running with account address: 0xFf89D90f8B653094DEeEDe762DdDf14E9c3073e6
{'status': 'ok', 'response': {'type': 'default'}}
{'status': 'ok', 'response': {'type': 'default'}}
